In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
NBA_learn_df=pd.read_csv("NBA_learning2.csv")
NBA_learn_df

Using TensorFlow backend.


,Unnamed: 0,Year,Age,G,GS,MP,PTS,PER,TS%,FTr,FG,FGA,AST,PF,height,weight,Players ok,Played_Years
0,0,1986,22.0,82,1.0,1542.0,521,11.8,0.564,0.430,209,388,54,229,203,106,0,0
1,1,1987,23.0,79,72.0,2240.0,852,15.7,0.599,0.480,316,587,84,171,203,106,0,1
2,2,1988,24.0,82,64.0,2636.0,937,14.5,0.581,0.592,322,640,93,204,203,106,0,2
3,3,1989,25.0,82,82.0,2510.0,1088,17.8,0.594,0.474,401,758,103,172,203,106,0,3
4,4,1990,26.0,82,82.0,2709.0,1061,14.7,0.548,0.459,385,806,90,207,203,106,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27369,24407,1968,31.0,82,NaN,3836.0,1992,24.7,0.557,0.677,819,1377,702,160,216,124,1,10
27370,24407,1968,31.0,82,NaN,3836.0,1992,24.7,0.557,0.677,819,1377,702,160,216,124,1,10
27371,24407,1968,31.0,82,NaN,3836.0,1992,24.7,0.557,0.677,819,1377,702,160,216,124,1,10
27372,24407,1968,31.0,82,NaN,3836.0,1992,24.7,0.557,0.677,819,1377,702,160,216,124,1,10


In [2]:
NBA_learn_df.sample(frac=1).reset_index(drop=True)

,Unnamed: 0,Year,Age,G,GS,MP,PTS,PER,TS%,FTr,FG,FGA,AST,PF,height,weight,Players ok,Played_Years
0,20260,1950,24.0,54,NaN,NaN,448,NaN,0.341,0.306,163,579,141,260,208,99,0,0
1,2110,1994,25.0,76,4.0,1726.0,588,13.0,0.510,0.252,233,519,263,132,185,81,0,3
2,23059,2009,32.0,77,3.0,1202.0,372,11.9,0.518,0.254,158,323,31,112,211,104,0,12
3,24479,1998,34.0,20,0.0,180.0,25,4.3,0.373,0.267,10,30,9,23,201,92,0,13
4,21916,1990,32.0,4,0.0,17.0,0,-21.8,0.000,0.000,0,2,1,4,203,99,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27369,23009,2013,26.0,49,0.0,913.0,399,12.3,0.522,0.193,139,352,93,103,188,88,0,4
27370,13897,2007,29.0,2,2.0,63.0,19,5.4,0.446,0.750,8,16,1,10,206,106,0,6
27371,24406,1967,30.0,81,NaN,3682.0,1956,26.5,0.637,0.761,785,1150,630,143,216,124,1,9
27372,1993,1985,32.0,67,37.0,1647.0,676,17.8,0.558,0.393,269,516,156,184,211,95,0,7


In [3]:
X=NBA_learn_df.drop(columns=["Unnamed: 0","Players ok"])
y=NBA_learn_df['Players ok']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

import keras
print(keras.__version__)

2.3.1


In [4]:
mod1 = Sequential() 
mod1.add(Dense(50, activation='relu',input_shape=(16,)))
mod1.add(Dropout(0.3))
mod1.add(Dense(100, activation='relu'))
#mod1.add(Dropout(0.3))
mod1.add(Dense(50, activation='relu'))
mod1.add(Dense(1, activation="sigmoid"))
mod1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                850       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 11,051
Trainable params: 11,051
Non-trainable params: 0
_________________________________________________________________


In [5]:
from keras.optimizers import SGD
from keras import metrics
from keras.metrics import Recall
opt = SGD(lr=0.0001)
#model.compile(loss = "categorical_crossentropy", optimizer = opt)
mod1.compile(loss='binary_crossentropy', optimizer = opt, metrics=['binary_accuracy',Recall(thresholds=[0.3])])#thresholds=[0.5,0.5,0.5],
checkpointer = ModelCheckpoint(filepath='NBA.h5', monitor='loss',verbose=0, save_best_only='min', period=1)
h = mod1.fit(X_train_scaled, encoded_y_train, batch_size=1000, epochs=100, verbose=1, validation_data = (X_test_scaled,encoded_y_test),callbacks=[checkpointer])

Train on 20530 samples, validate on 6844 samples
Epoch 1/100
20530/20530 [==============================] - 1s 47us/step - loss: nan - binary_accuracy: 0.0033 - recall_1: 0.0309 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 5/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.00

20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 40/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 41/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 42/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 43/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - va

20530/20530 [==============================] - 0s 9us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 78/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 79/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 80/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - val_binary_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 81/100
20530/20530 [==============================] - 0s 8us/step - loss: nan - binary_accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: nan - va

In [6]:
model_loss, model_accuracy,recall = mod1.evaluate(
    X_test_scaled, encoded_y_test, verbose=0)
print("Normal Neural Network - Loss:" +str(model_loss)+", Accuracy: "+str(model_accuracy)+ " Recall: "+str(recall))
#mod1.evaluate(    X_test_scaled, encoded_y_test, verbose=0)

Normal Neural Network - Loss:nan, Accuracy: 0.0 Recall: 0.0


In [7]:
encoded_predictions = mod1.predict_classes(X_test_scaled[:1000])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
print("Predicted classes:"+ str(prediction_labels))
print("Actual Labels: "+str(list(y_test[:1000])))

Predicted classes:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 

/Users/henrigaspard/opt/anaconda3/envs/qqdata/lib/python3.7/site-packages/keras/engine/sequential.py:272: RuntimeWarning: invalid value encountered in greater
  return (proba > 0.5).astype('int32')
/Users/henrigaspard/opt/anaconda3/envs/qqdata/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
# save the model to disk
# tfjs.converters.save_keras_model(mod1, './keras_model')
# filename = 'finalized_model.pkl'
# pickle.dump(mod1, open(filename, 'wb'))

In [9]:
NBA_learn_df=NBA_learn_df.dropna()
X=NBA_learn_df.drop(columns=["Unnamed: 0","PTS"])
y=NBA_learn_df['PTS']

In [10]:
X_scaler = MinMaxScaler().fit(X)
X_scaled = X_scaler.transform(X)
y=pd.DataFrame(y)
y_scaler = MinMaxScaler().fit(y)#.reshape(-1, 1)
y_scaled = y_scaler.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, random_state=1)

In [11]:
mod2 = Sequential() 
mod2.add(Dense(50, activation='relu',input_shape=(16,)))
#mod2.add(Dropout(0.3))
mod2.add(Dense(100, activation='relu'))
#mod2.add(Dropout(0.3))
mod2.add(Dense(50, activation='relu'))
mod2.add(Dense(1, activation="relu"))
mod2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 50)                850       
_________________________________________________________________
dense_6 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 11,051
Trainable params: 11,051
Non-trainable params: 0
_________________________________________________________________


In [12]:
from keras import metrics
#from keras.metrics import MeanRelativeError
opt = SGD(lr=0.1)
mod2.compile(loss="mse", optimizer = opt,metrics=["mean_absolute_percentage_error"])#, metrics=['accuracy']
checkpointer = ModelCheckpoint(filepath='NBA2.h5', monitor='loss',verbose=0, save_best_only='min', period=1)
h = mod2.fit(X_train, y_train, batch_size=1000, epochs=1000, verbose=1, validation_data = (X_test,y_test),callbacks=[checkpointer])

Train on 14922 samples, validate on 4974 samples
Epoch 1/1000
14922/14922 [==============================] - 1s 37us/step - loss: 0.0141 - mean_absolute_percentage_error: 522032.0312 - val_loss: 0.0037 - val_mean_absolute_percentage_error: 260662.2031
Epoch 2/1000
14922/14922 [==============================] - 0s 5us/step - loss: 0.0029 - mean_absolute_percentage_error: 209703.4219 - val_loss: 0.0023 - val_mean_absolute_percentage_error: 141758.1719
Epoch 3/1000
14922/14922 [==============================] - 0s 5us/step - loss: 0.0020 - mean_absolute_percentage_error: 163285.4219 - val_loss: 0.0018 - val_mean_absolute_percentage_error: 125804.3125
Epoch 4/1000
14922/14922 [==============================] - 0s 6us/step - loss: 0.0017 - mean_absolute_percentage_error: 155693.6562 - val_loss: 0.0015 - val_mean_absolute_percentage_error: 109539.8750
Epoch 5/1000
14922/14922 [==============================] - 0s 5us/step - loss: 0.0015 - mean_absolute_percentage_error: 149229.1250 - val_los

14922/14922 [==============================] - 0s 6us/step - loss: 4.3133e-04 - mean_absolute_percentage_error: 21459.6934 - val_loss: 4.2602e-04 - val_mean_absolute_percentage_error: 21506.9141
Epoch 41/1000
14922/14922 [==============================] - 0s 5us/step - loss: 4.3429e-04 - mean_absolute_percentage_error: 21012.1895 - val_loss: 4.1988e-04 - val_mean_absolute_percentage_error: 24964.5977
Epoch 42/1000
14922/14922 [==============================] - 0s 5us/step - loss: 4.2069e-04 - mean_absolute_percentage_error: 19673.6094 - val_loss: 4.1460e-04 - val_mean_absolute_percentage_error: 20477.1738
Epoch 43/1000
14922/14922 [==============================] - 0s 5us/step - loss: 4.1739e-04 - mean_absolute_percentage_error: 18337.1875 - val_loss: 4.2760e-04 - val_mean_absolute_percentage_error: 17144.9883
Epoch 44/1000
14922/14922 [==============================] - 0s 5us/step - loss: 4.3585e-04 - mean_absolute_percentage_error: 18573.1016 - val_loss: 4.0955e-04 - val_mean_absolut

14922/14922 [==============================] - 0s 6us/step - loss: 3.3680e-04 - mean_absolute_percentage_error: 3597.4070 - val_loss: 3.2930e-04 - val_mean_absolute_percentage_error: 10275.2568
Epoch 80/1000
14922/14922 [==============================] - 0s 6us/step - loss: 3.3544e-04 - mean_absolute_percentage_error: 3942.6838 - val_loss: 3.4087e-04 - val_mean_absolute_percentage_error: 11715.5293
Epoch 81/1000
14922/14922 [==============================] - 0s 5us/step - loss: 3.3684e-04 - mean_absolute_percentage_error: 3736.6108 - val_loss: 3.2533e-04 - val_mean_absolute_percentage_error: 10634.5010
Epoch 82/1000
14922/14922 [==============================] - 0s 5us/step - loss: 3.2897e-04 - mean_absolute_percentage_error: 3592.4185 - val_loss: 3.2589e-04 - val_mean_absolute_percentage_error: 10864.9268
Epoch 83/1000
14922/14922 [==============================] - 0s 5us/step - loss: 3.4481e-04 - mean_absolute_percentage_error: 4104.5161 - val_loss: 3.2316e-04 - val_mean_absolute_per

14922/14922 [==============================] - 0s 5us/step - loss: 2.9012e-04 - mean_absolute_percentage_error: 1862.7469 - val_loss: 2.8437e-04 - val_mean_absolute_percentage_error: 9289.9072
Epoch 119/1000
14922/14922 [==============================] - 0s 5us/step - loss: 3.2004e-04 - mean_absolute_percentage_error: 1982.7853 - val_loss: 2.9980e-04 - val_mean_absolute_percentage_error: 9778.7041
Epoch 120/1000
14922/14922 [==============================] - 0s 5us/step - loss: 2.8724e-04 - mean_absolute_percentage_error: 1959.2354 - val_loss: 2.9925e-04 - val_mean_absolute_percentage_error: 9038.4854
Epoch 121/1000
14922/14922 [==============================] - 0s 5us/step - loss: 2.8428e-04 - mean_absolute_percentage_error: 1822.3656 - val_loss: 2.8442e-04 - val_mean_absolute_percentage_error: 9626.2539
Epoch 122/1000
14922/14922 [==============================] - 0s 5us/step - loss: 2.9132e-04 - mean_absolute_percentage_error: 1704.6686 - val_loss: 3.0058e-04 - val_mean_absolute_per

14922/14922 [==============================] - 0s 5us/step - loss: 2.8056e-04 - mean_absolute_percentage_error: 1435.3502 - val_loss: 2.8545e-04 - val_mean_absolute_percentage_error: 8694.7715
Epoch 158/1000
14922/14922 [==============================] - 0s 6us/step - loss: 2.6247e-04 - mean_absolute_percentage_error: 1257.1139 - val_loss: 2.6713e-04 - val_mean_absolute_percentage_error: 9656.2715
Epoch 159/1000
14922/14922 [==============================] - 0s 5us/step - loss: 2.5903e-04 - mean_absolute_percentage_error: 1185.4769 - val_loss: 2.5804e-04 - val_mean_absolute_percentage_error: 9607.7959
Epoch 160/1000
14922/14922 [==============================] - 0s 6us/step - loss: 2.7216e-04 - mean_absolute_percentage_error: 1190.1122 - val_loss: 2.5158e-04 - val_mean_absolute_percentage_error: 9072.6777
Epoch 161/1000
14922/14922 [==============================] - 0s 6us/step - loss: 2.6535e-04 - mean_absolute_percentage_error: 960.8477 - val_loss: 2.6438e-04 - val_mean_absolute_perc

...

14922/14922 [==============================] - 0s 5us/step - loss: 1.7803e-04 - mean_absolute_percentage_error: 1602.6692 - val_loss: 1.7638e-04 - val_mean_absolute_percentage_error: 9822.3965
Epoch 821/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.6179e-04 - mean_absolute_percentage_error: 1476.6979 - val_loss: 1.5411e-04 - val_mean_absolute_percentage_error: 9149.4346
Epoch 822/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.5746e-04 - mean_absolute_percentage_error: 1536.4927 - val_loss: 1.5161e-04 - val_mean_absolute_percentage_error: 8804.2217
Epoch 823/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.6185e-04 - mean_absolute_percentage_error: 1595.3973 - val_loss: 1.5379e-04 - val_mean_absolute_percentage_error: 8681.4062
Epoch 824/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.6853e-04 - mean_absolute_percentage_error: 1379.1287 - val_loss: 1.5140e-04 - val_mean_absolute_per

14922/14922 [==============================] - 0s 5us/step - loss: 1.7840e-04 - mean_absolute_percentage_error: 1668.0720 - val_loss: 1.6669e-04 - val_mean_absolute_percentage_error: 8480.6318
Epoch 860/1000
14922/14922 [==============================] - 0s 5us/step - loss: 2.2196e-04 - mean_absolute_percentage_error: 1285.3829 - val_loss: 2.3870e-04 - val_mean_absolute_percentage_error: 10752.4902
Epoch 861/1000
14922/14922 [==============================] - 0s 5us/step - loss: 2.1612e-04 - mean_absolute_percentage_error: 1234.7117 - val_loss: 1.7388e-04 - val_mean_absolute_percentage_error: 8347.4854
Epoch 862/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.7120e-04 - mean_absolute_percentage_error: 1381.2695 - val_loss: 1.5475e-04 - val_mean_absolute_percentage_error: 9239.6494
Epoch 863/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.5605e-04 - mean_absolute_percentage_error: 1460.1440 - val_loss: 1.4890e-04 - val_mean_absolute_pe

14922/14922 [==============================] - 0s 5us/step - loss: 1.5916e-04 - mean_absolute_percentage_error: 1555.1854 - val_loss: 1.6271e-04 - val_mean_absolute_percentage_error: 8407.5479
Epoch 899/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.7393e-04 - mean_absolute_percentage_error: 1311.0782 - val_loss: 1.7891e-04 - val_mean_absolute_percentage_error: 9940.8799
Epoch 900/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.9054e-04 - mean_absolute_percentage_error: 1348.9858 - val_loss: 2.2109e-04 - val_mean_absolute_percentage_error: 8030.6328
Epoch 901/1000
14922/14922 [==============================] - 0s 5us/step - loss: 2.3432e-04 - mean_absolute_percentage_error: 1723.3663 - val_loss: 1.8906e-04 - val_mean_absolute_percentage_error: 9961.2275
Epoch 902/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.8678e-04 - mean_absolute_percentage_error: 1267.5491 - val_loss: 1.9659e-04 - val_mean_absolute_per

14922/14922 [==============================] - 0s 9us/step - loss: 1.5322e-04 - mean_absolute_percentage_error: 1525.4314 - val_loss: 1.4471e-04 - val_mean_absolute_percentage_error: 8845.8496
Epoch 938/1000
14922/14922 [==============================] - 0s 6us/step - loss: 1.6239e-04 - mean_absolute_percentage_error: 1568.9722 - val_loss: 1.5913e-04 - val_mean_absolute_percentage_error: 8398.1904
Epoch 939/1000
14922/14922 [==============================] - 0s 6us/step - loss: 1.6590e-04 - mean_absolute_percentage_error: 1562.4263 - val_loss: 1.5910e-04 - val_mean_absolute_percentage_error: 9508.0996
Epoch 940/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.5090e-04 - mean_absolute_percentage_error: 1448.6001 - val_loss: 1.4353e-04 - val_mean_absolute_percentage_error: 8713.3467
Epoch 941/1000
14922/14922 [==============================] - 0s 6us/step - loss: 1.4988e-04 - mean_absolute_percentage_error: 1504.9736 - val_loss: 1.4517e-04 - val_mean_absolute_per

14922/14922 [==============================] - 0s 5us/step - loss: 1.5066e-04 - mean_absolute_percentage_error: 1584.2872 - val_loss: 1.6349e-04 - val_mean_absolute_percentage_error: 9668.8604
Epoch 977/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.6766e-04 - mean_absolute_percentage_error: 1655.7198 - val_loss: 1.6807e-04 - val_mean_absolute_percentage_error: 8253.3506
Epoch 978/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.7475e-04 - mean_absolute_percentage_error: 1833.7814 - val_loss: 2.1120e-04 - val_mean_absolute_percentage_error: 10337.5215
Epoch 979/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.9286e-04 - mean_absolute_percentage_error: 1290.0588 - val_loss: 1.5915e-04 - val_mean_absolute_percentage_error: 8321.3281
Epoch 980/1000
14922/14922 [==============================] - 0s 5us/step - loss: 1.5068e-04 - mean_absolute_percentage_error: 1351.6968 - val_loss: 1.4100e-04 - val_mean_absolute_pe

In [13]:
model_loss, model_accuracy = mod2.evaluate(
    X_test, y_test, verbose=0)
print("Normal Neural Network - val_loss:" +str(model_loss)+", val_mean_absolute_percentage_error: "+str(model_accuracy))

Normal Neural Network - val_loss:0.00014614715426624294, val_mean_absolute_percentage_error: 9275.654296875


In [14]:
mod2.evaluate(X_test, y_test, verbose=0)

[0.00014614715426624294, 9275.654296875]

In [15]:
# save the model 2 to disk
filename2 = 'finalized_model_2.pkl'
pickle.dump(mod2, open(filename2, 'wb'))

NameError: name 'pickle' is not defined